In [1]:
import os
import numpy as np
import cv2

In [2]:
y = np.load("train_labels.npy")
X = np.load("normalized_train_rgb_data.npy")

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available: 1


In [4]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

In [5]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
output = Dense(10, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = output)

#Freeze first layers
for layer in base_model.layers[:20]:
    layer.trainable = False
    
#Unfreeze the remaining layers to allow fine-tuning
for layer in base_model.layers[20:]:
    layer.trainable = True

In [6]:
from keras.optimizers import Adam
from keras import metrics
adam = Adam(learning_rate=0.0001)
model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy', metrics.Precision(), metrics.Recall()])

In [7]:
history = model.fit(X, y, epochs=10, batch_size=512, validation_split=0.1)

Epoch 1/20
87/88 [============================>.] - ETA: 0s - loss: 2.6523 - accuracy: 0.1527 - precision: 0.1285 - recall: 0.0106

In [ ]:
model.save("WildSeekAI.keras")